In [1]:
import numpy as np
import pandas as pd

In [2]:


building_df = pd.read_csv('ashrae-energy-prediction/building_metadata.csv')
weather_train = pd.read_csv("ashrae-energy-prediction/weather_train.csv")
train = pd.read_csv("ashrae-energy-prediction/train.csv")
train = train.merge(building_df, on = "building_id", how = "outer")
train = train.merge(weather_train, on = ["site_id", "timestamp"],how = "outer")
del weather_train

In [3]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0.0,0.0,2016-01-01 00:00:00,0.0,0,Education,7432.0,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1.0,0.0,2016-01-01 00:00:00,0.0,0,Education,2720.0,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2.0,0.0,2016-01-01 00:00:00,0.0,0,Education,5376.0,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3.0,0.0,2016-01-01 00:00:00,0.0,0,Education,23685.0,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4.0,0.0,2016-01-01 00:00:00,0.0,0,Education,116607.0,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [4]:
def average_imputation(df, column_name):
    imputation = df.groupby(['timestamp'])[column_name].mean()
    
    df.loc[df[column_name].isnull(), column_name] = df[df[column_name].isnull()][[column_name]].apply(lambda x: imputation[df['timestamp'][x.index]].values)
    del imputation
    return df

In [5]:
train = average_imputation(train, 'wind_speed')
train = average_imputation(train, 'wind_direction')
train = average_imputation(train, 'air_temperature')
train = average_imputation(train,'sea_level_pressure')
train = average_imputation(train,'dew_temperature')

In [6]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0.0,0.0,2016-01-01 00:00:00,0.0,0,Education,7432.0,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1.0,0.0,2016-01-01 00:00:00,0.0,0,Education,2720.0,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2.0,0.0,2016-01-01 00:00:00,0.0,0,Education,5376.0,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3.0,0.0,2016-01-01 00:00:00,0.0,0,Education,23685.0,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4.0,0.0,2016-01-01 00:00:00,0.0,0,Education,116607.0,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [7]:
del train["timestamp"]

In [8]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0.0,0.0,0.0,0,Education,7432.0,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1.0,0.0,0.0,0,Education,2720.0,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2.0,0.0,0.0,0,Education,5376.0,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3.0,0.0,0.0,0,Education,23685.0,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4.0,0.0,0.0,0,Education,116607.0,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [91]:
train['floor_count'].value_counts

<bound method IndexOpsMixin.value_counts of 0           1.0
1           1.0
2           1.0
3           1.0
4           1.0
           ... 
20216230    1.0
20216231    1.0
20216232    1.0
20216233    1.0
20216234    1.0
Name: floor_count, Length: 20216235, dtype: float64>

In [9]:
train['primary_use'].value_counts

<bound method IndexOpsMixin.value_counts of 0           Education
1           Education
2           Education
3           Education
4           Education
              ...    
20216230          NaN
20216231          NaN
20216232          NaN
20216233          NaN
20216234          NaN
Name: primary_use, Length: 20216235, dtype: object>

In [10]:
train.isna().sum()

building_id                135
meter                      135
meter_reading              135
site_id                      0
primary_use                135
square_feet                135
year_built            12127780
floor_count           16709302
air_temperature              0
cloud_coverage         8825441
dew_temperature              0
precip_depth_1_hr      3749121
sea_level_pressure           0
wind_direction               0
wind_speed                   0
dtype: int64

In [11]:
#train['building_id'] = train['building_id'].ffill(axis = 0)
#train['meter'] = train['meter'].ffill(axis = 0)
#train['meter_reading'] = train['meter_reading'].ffill(axis = 0)

In [12]:
train.isna().sum()

building_id                  0
meter                        0
meter_reading                0
site_id                      0
primary_use                135
square_feet                135
year_built            12127780
floor_count           16709302
air_temperature              0
cloud_coverage         8825441
dew_temperature              0
precip_depth_1_hr      3749121
sea_level_pressure           0
wind_direction               0
wind_speed                   0
dtype: int64

In [13]:
train['primary_use'] = train['primary_use'].ffill(axis = 0)

In [14]:
train.isna().sum()

building_id                  0
meter                        0
meter_reading                0
site_id                      0
primary_use                  0
square_feet                135
year_built            12127780
floor_count           16709302
air_temperature              0
cloud_coverage         8825441
dew_temperature              0
precip_depth_1_hr      3749121
sea_level_pressure           0
wind_direction               0
wind_speed                   0
dtype: int64

In [15]:
train['square_feet'] = train['square_feet'].ffill(axis = 0)

In [16]:
train.isna().sum()

building_id                  0
meter                        0
meter_reading                0
site_id                      0
primary_use                  0
square_feet                  0
year_built            12127780
floor_count           16709302
air_temperature              0
cloud_coverage         8825441
dew_temperature              0
precip_depth_1_hr      3749121
sea_level_pressure           0
wind_direction               0
wind_speed                   0
dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train["primary_use"] = le.fit_transform(train.primary_use)

categoricals = ["site_id", "building_id", "primary_use", "hour", "weekday", "meter",  "wind_direction"]

In [97]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0.0,0.0,0.0,0,0,7432.0,2008.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
1,1.0,0.0,0.0,0,0,2720.0,2004.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
2,2.0,0.0,0.0,0,0,5376.0,1991.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
3,3.0,0.0,0.0,0,0,23685.0,2002.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
4,4.0,0.0,0.0,0,0,116607.0,1975.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0


In [19]:
train['floor_count'].isnull().sum()

16709302

In [95]:
train["floor_count"].value_counts()

1.0     17643605
2.0       691052
4.0       351938
3.0       331688
6.0       296598
5.0       253856
8.0       175258
7.0       131435
9.0        86416
11.0       61465
13.0       43911
10.0       34880
21.0       26343
19.0       26343
26.0       26321
12.0       17559
16.0        8784
14.0        8783
Name: floor_count, dtype: int64

In [21]:
train.shape

(20216235, 15)

In [22]:
val = train['floor_count'].value_counts().index[0]

In [23]:
val

1.0

In [24]:
train['floor_count'].fillna(val,inplace=True)

In [25]:
train["floor_count"].isnull().sum()

0

In [26]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0.0,0.0,0.0,0,0,7432.0,2008.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1.0,0.0,0.0,0,0,2720.0,2004.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2.0,0.0,0.0,0,0,5376.0,1991.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3.0,0.0,0.0,0,0,23685.0,2002.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4.0,0.0,0.0,0,0,116607.0,1975.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [27]:
train['precip_depth_1_hr'].value_counts

<bound method IndexOpsMixin.value_counts of 0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
            ..
20216230   NaN
20216231   NaN
20216232   NaN
20216233   NaN
20216234   NaN
Name: precip_depth_1_hr, Length: 20216235, dtype: float64>

In [28]:
#train.drop(['precip_depth_1_hr'],axis = 1)

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure,wind_direction,wind_speed
0,0.0,0.0,0.0000,0,0,7432.0,2008.0,1.0,25.0,6.0,20.0,1019.7,0.0,0.0
1,1.0,0.0,0.0000,0,0,2720.0,2004.0,1.0,25.0,6.0,20.0,1019.7,0.0,0.0
2,2.0,0.0,0.0000,0,0,5376.0,1991.0,1.0,25.0,6.0,20.0,1019.7,0.0,0.0
3,3.0,0.0,0.0000,0,0,23685.0,2002.0,1.0,25.0,6.0,20.0,1019.7,0.0,0.0
4,4.0,0.0,0.0000,0,0,116607.0,1975.0,1.0,25.0,6.0,20.0,1019.7,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216230,1400.0,1.0,16.3553,15,4,21168.0,NaN,1.0,0.0,4.0,-4.4,1015.2,300.0,8.8
20216231,1400.0,1.0,16.3553,15,4,21168.0,NaN,1.0,-0.6,NaN,-5.0,1017.3,310.0,7.2
20216232,1400.0,1.0,16.3553,15,4,21168.0,NaN,1.0,-0.6,NaN,-5.0,1018.2,310.0,8.2
20216233,1400.0,1.0,16.3553,15,4,21168.0,NaN,1.0,0.0,NaN,-5.6,1019.1,300.0,10.3


In [29]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0.0,0.0,0.0,0,0,7432.0,2008.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1.0,0.0,0.0,0,0,2720.0,2004.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2.0,0.0,0.0,0,0,5376.0,1991.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3.0,0.0,0.0,0,0,23685.0,2002.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4.0,0.0,0.0,0,0,116607.0,1975.0,1.0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [ ]:
train.co

In [30]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [31]:
val1 = train['year_built'].value_counts().index[0]

In [32]:
val1

1976.0

In [33]:
train['year_built'].fillna(val1,inplace=True)

In [34]:
train["year_built"].isnull().sum()

0

In [35]:
train.isna().sum()

building_id                 0
meter                       0
meter_reading               0
site_id                     0
primary_use                 0
square_feet                 0
year_built                  0
floor_count                 0
air_temperature             0
cloud_coverage        8825441
dew_temperature             0
precip_depth_1_hr     3749121
sea_level_pressure          0
wind_direction              0
wind_speed                  0
dtype: int64

In [36]:
val2 = train['cloud_coverage'].value_counts().index[1]

In [37]:
val2

2.0

In [38]:
train['cloud_coverage'].fillna(val2,inplace=True)

In [39]:
val3 = train['precip_depth_1_hr'].value_counts().index[1]

In [40]:
val3

-1.0

In [41]:
train['precip_depth_1_hr'].fillna(val3,inplace=True)

In [42]:
train.isna().sum()

building_id           0
meter                 0
meter_reading         0
site_id               0
primary_use           0
square_feet           0
year_built            0
floor_count           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [53]:
Y = train['meter_reading']
Y.head()
X = train.drop(['meter_reading','primary_use','square_feet','year_built','floor_count','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed'],axis = 1)

In [54]:
X.head()

,building_id,meter,site_id,air_temperature
0,0.0,0.0,0,25.0
1,1.0,0.0,0,25.0
2,2.0,0.0,0,25.0
3,3.0,0.0,0,25.0
4,4.0,0.0,0,25.0


In [55]:
Y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: meter_reading, dtype: float64

In [56]:
lr.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [57]:
lr.intercept_

2840.6233903868665

In [58]:
lr.coef_

array([ -24.47556285, 2285.90252098, 2238.57090625,  -35.41776216])

In [59]:
building_df = pd.read_csv('ashrae-energy-prediction/building_metadata.csv')
weather_test = pd.read_csv("ashrae-energy-prediction/weather_test.csv")
test = pd.read_csv("ashrae-energy-prediction/test.csv")
test = test.merge(building_df, on = "building_id", how = "outer")
test = test.merge(weather_test, on = ["site_id", "timestamp"],how = "outer")
del weather_test

In [60]:
test.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,2008.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,2004.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,1991.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,2002.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,1975.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6


In [61]:
test.count

<bound method DataFrame.count of             row_id  building_id  meter            timestamp  site_id  \
0                0            0      0  2017-01-01 00:00:00        0   
1                1            1      0  2017-01-01 00:00:00        0   
2                2            2      0  2017-01-01 00:00:00        0   
3                3            3      0  2017-01-01 00:00:00        0   
4                4            4      0  2017-01-01 00:00:00        0   
...            ...          ...    ...                  ...      ...   
41697595  41697595         1444      0  2018-05-09 07:00:00       15   
41697596  41697596         1445      0  2018-05-09 07:00:00       15   
41697597  41697597         1446      0  2018-05-09 07:00:00       15   
41697598  41697598         1447      0  2018-05-09 07:00:00       15   
41697599  41697599         1448      0  2018-05-09 07:00:00       15   

                            primary_use  square_feet  year_built  floor_count  \
0                    

In [62]:
def average_imputation(df, column_name):
    imputation = df.groupby(['timestamp'])[column_name].mean()
    
    df.loc[df[column_name].isnull(), column_name] = df[df[column_name].isnull()][[column_name]].apply(lambda x: imputation[df['timestamp'][x.index]].values)
    del imputation
    return df

In [63]:
test = average_imputation(test, 'wind_speed')
test = average_imputation(test, 'wind_direction')
test = average_imputation(test, 'air_temperature')
test = average_imputation(test,'sea_level_pressure')
test = average_imputation(test,'dew_temperature')

In [64]:
del test["timestamp"]

In [65]:
test.head()

,row_id,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,0,Education,7432,2008.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,1,1,0,0,Education,2720,2004.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
2,2,2,0,0,Education,5376,1991.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
3,3,3,0,0,Education,23685,2002.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
4,4,4,0,0,Education,116607,1975.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6


In [66]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
test["primary_use"] = le.fit_transform(test["primary_use"])

categoricals = ["site_id", "building_id", "primary_use", "hour", "weekday", "meter",  "wind_direction"]

In [67]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0.0,0.0,0.0,0,0,7432.0,2008.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
1,1.0,0.0,0.0,0,0,2720.0,2004.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
2,2.0,0.0,0.0,0,0,5376.0,1991.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
3,3.0,0.0,0.0,0,0,23685.0,2002.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
4,4.0,0.0,0.0,0,0,116607.0,1975.0,1.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0


In [68]:
test['floor_count'].isnull().sum()
test["floor_count"].value_counts()
val = test['floor_count'].value_counts().index[0]
val
test['floor_count'].fillna(val,inplace=True)
test["floor_count"].isnull().sum()
test.drop(['precip_depth_1_hr'],axis = 1)
val4 = test['year_built'].value_counts().index[0]
test['year_built'].fillna(val4,inplace=True)
val5 = test['cloud_coverage'].value_counts().index[1]
test['cloud_coverage'].fillna(val5,inplace=True)
val6 = test['precip_depth_1_hr'].value_counts().index[1]
test['precip_depth_1_hr'].fillna(val6,inplace=True)
test.isna().sum()

row_id                0
building_id           0
meter                 0
site_id               0
primary_use           0
square_feet           0
year_built            0
floor_count           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [71]:
test.head()

,row_id,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,0,0,7432,2008.0,1.0,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
1,1,1,0,0,0,2720,2004.0,1.0,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
2,2,2,0,0,0,5376,1991.0,1.0,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
3,3,3,0,0,0,23685,2002.0,1.0,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
4,4,4,0,0,0,116607,1975.0,1.0,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6


In [72]:
test.count

<bound method DataFrame.count of             row_id  building_id  meter  site_id  primary_use  square_feet  \
0                0            0      0        0            0         7432   
1                1            1      0        0            0         2720   
2                2            2      0        0            0         5376   
3                3            3      0        0            0        23685   
4                4            4      0        0            0       116607   
...            ...          ...    ...      ...          ...          ...   
41697595  41697595         1444      0       15            1        19619   
41697596  41697596         1445      0       15            0         4298   
41697597  41697597         1446      0       15            1        11265   
41697598  41697598         1447      0       15            4        29775   
41697599  41697599         1448      0       15            6        92271   

          year_built  floor_count  air_tem

In [73]:
row_id = test['row_id']

In [74]:
row_id.head()

0    0
1    1
2    2
3    3
4    4
Name: row_id, dtype: int64

In [75]:
test.drop(["row_id"], axis = 1, inplace = True)

In [76]:
train.shape

(20216235, 15)

In [81]:
test.head()
test = test.drop(['primary_use','square_feet','year_built','floor_count','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed'],axis = 1)
#predictions = lr.predict(test)

In [82]:
predictions = lr.predict(test)

In [83]:
predictions

array([2210.1872239 , 2185.71166105, 2161.23609821, ...,  418.86138459,
        394.38582175,  369.9102589 ])

In [84]:
submission = pd.DataFrame({'row_id':row_id,'meter_reading':predictions})

In [85]:
submission.head()

,row_id,meter_reading
0,0,2210.187224
1,1,2185.711661
2,2,2161.236098
3,3,2136.760535
4,4,2112.284973


In [86]:
submission.count

<bound method DataFrame.count of             row_id  meter_reading
0                0    2210.187224
1                1    2185.711661
2                2    2161.236098
3                3    2136.760535
4                4    2112.284973
...            ...            ...
41697595  41697595     467.812510
41697596  41697596     443.336947
41697597  41697597     418.861385
41697598  41697598     394.385822
41697599  41697599     369.910259

[41697600 rows x 2 columns]>

In [87]:
filename = 'ASHRAE Predictions2.csv'

In [88]:
submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: ASHRAE Predictions2.csv
